In [4]:
# Настройка Python 3.8 в Google Colab
!bash ./Miniconda3-py38_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py38_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py38_0
    - cffi==1.14.0=py38h2e261b9_0
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.0=py38h7b6447c_0
    - conda==4.8.2=py38_0
    - cryptography==2.8=py38h1ba5d50_0
    - idna==2.8=py38_1000
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20181209=hc058e9b_0
    - libffi==3.2.1=hd88cf55_4
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_0
    - openssl==1.1.1d=h7b6447c_4
    - pip==20.0.2=py38_1
    - pycosat==0.6.3=py38h7b6447c_0
    - pycparser==2.19=py_0
    - pyopenssl==19.1.0=py38_0
    - pysocks==1.7.1=py38_0
    - python==3.8.1=h0371630_1
    - readline==7.0=h7b6447c_5
    - requests==2.22.0=py38_1
    - ruamel_yaml==0.15.

In [1]:
import sys
print(sys.version)

3.8.17 (default, Jul  5 2023, 20:41:08) 
[GCC 11.2.0]


In [3]:
!pip install -r ./requirements.txt

In [6]:
!pip install seaborn
!pip install scikit-learn
!pip install protobuf==3.20.*

     |████████████████████████████████| 293 kB 6.5 MB/s 


In [2]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
import imgaug

2023-07-20 14:09:57.193086: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
# Корневой каталог проекта
ROOT_DIR = "/content/drive/MyDrive/test_work"

# Импорт Mask RCNN
sys.path.append(ROOT_DIR)  # путь до локальной версии библиотеки
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Путь к файлу тренировочных весов
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Каталог для сохранения журналов и контрольных точек модели, если он не указан
# через аргумент командной строки --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [4]:
class CustomConfig(Config):
    """
    Конфигурация для обучения на пользовательском наборе данных.
    Унаследован от базового класса Config и переопределяет некоторые значения.
    """
    # Дайте конфигурации узнаваемое имя
    NAME = "object"


    # КОЛИЧЕСТВО графических процессоров для использования. При использовании только ЦП необходимо установить значение 1.
    GPU_COUNT = 1

    # Мы используем графический процессор с 12 ГБ памяти, который может вместить два изображения.
    # Уменьшите значение, если вы используете меньший графический процессор.

    IMAGES_PER_GPU = 1

    # Количество классов вклюяая фон
    NUM_CLASSES = 1 + 1  # Фон + Рельсы

    # Количество шагов обучения в эпоху

    STEPS_PER_EPOCH = 5

    # Пропустить обнаружения с < 90% точностью
    DETECTION_MIN_CONFIDENCE = 0.9

    LEARNING_RATE = 0.001

In [5]:
############################################################
#  Dataset
############################################################

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset, jsons, ns):
        """ 
        dataset_dir: корневой каталог набора данных.
        subset: Подмножество для загрузки: train или val
        jsons: путь до JSON-файла с разметкой
        ns: Наименование меток в JSON
        """
        
        # Добавление классов
        self.add_class("object", 1, "Rails")


        dataset_dir = os.path.join(dataset_dir, subset)

        # Загрзука аннотации
        # VGG Image Annotator сохраняет каждое изображение в виде:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # В основном нас интересуют координаты x и y каждой области
        annotations1 = json.load(open(jsons))
        annotations = list(annotations1.values())  # ключи не нужны

        # The VIA tool сохраняет изображения в JSON, даже если они не имеют аннотаций 
        # Пропустить неаннотированные изображения.
        annotations = [a for a in annotations if a['regions']]

        # Добавиляем изображение
        for a in annotations:
            # Получаем координаты x, y точек многоугольников, составляющих
            # схема каждого экземпляра объекта. В
            # shape_attributes есть хранилища (смотрите формат json выше)
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes'][ns] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"Rails": 1}

            num_ids = [name_dict[a] for a in objects]

            # load_mask() нужен размер изображения, чтобы преобразовать полигоны в маски.
            # К сожалению, VIA не включает его в JSON, поэтому мы должны прочитать
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  # для одного класса просто добавьте название здесь
                image_id=a['filename'],  # используйте имя файла в качестве уникального идентификатора изображения
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Создание масок экземпляра для изображения.
       Returns:
        masks: Массив bool формы [height, width, instance count] с одной маской на экземпляр.
        class_ids: одномерный массив идентификаторов классов масок экземпляра.
        """
        
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Преобразуйте полигоны в растровую маску формы
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Получите индексы пикселей внутри многоугольника и установите их равными 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Возвращает маску и массив идентификаторов классов каждого экземпляра. С тех пор как у нас есть
        # только один идентификатор класса, мы возвращаем массив из 1s
        # Сопоставьте имена классов с идентификаторами классов.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Возвращаем путь к изображению."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [8]:
def train(model):
    """Обучение модели."""
    # Обучающий набор данных.
    dataset_train = CustomDataset()
    dataset_train.load_custom("/content/drive/MyDrive/test_work/datasets/", "train", '/content/drive/MyDrive/test_work/datasets/train/train.json', 'Rails')
    dataset_train.prepare()

    # Проверочный набор данных
    dataset_val = CustomDataset()
    dataset_val.load_custom("/content/drive/MyDrive/test_work/datasets/", "val", '/content/drive/MyDrive/test_work/datasets/val/val.json', 'names')
    dataset_val.prepare()

    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=250,
                layers='heads')

    # Аугментация изображений
    # model.train(dataset_train, dataset_val,
    #             learning_rate=config.LEARNING_RATE,
    #             epochs=300,
    #             layers='heads', #layers='all',
    #             augmentation = imgaug.augmenters.Sequential([
    #             imgaug.augmenters.Fliplr(1),
    #             imgaug.augmenters.Flipud(1),
    #             imgaug.augmenters.Affine(rotate=(-45, 45)),
    #             imgaug.augmenters.Affine(rotate=(-90, 90)),
    #             imgaug.augmenters.Affine(scale=(0.5, 1.5)),
    #             imgaug.augmenters.Crop(px=(0, 10)),
    #             imgaug.augmenters.Grayscale(alpha=(0.0, 1.0)),
    #             imgaug.augmenters.AddToHueAndSaturation((-20, 20)), # change hue and saturation
    #             imgaug.augmenters.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
    #             imgaug.augmenters.Invert(0.05, per_channel=True), # invert color channels
    #             imgaug.augmenters.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images

    #             ]

    #             ))

# Другой способ аугментации
# augmentation = imgaug.Sometimes(5/6,aug.OneOf(
                                            # [
                                            # imgaug.augmenters.Fliplr(1),
                                            # imgaug.augmenters.Flipud(1),
                                            # imgaug.augmenters.Affine(rotate=(-45, 45)),
                                            # imgaug.augmenters.Affine(rotate=(-90, 90)),
                                            # imgaug.augmenters.Affine(scale=(0.5, 1.5))
                                             # ]
                                        # )
                                   # )

'\n this augmentation is applied consecutively to each image. In other words, for each image, the augmentation apply flip LR,\n and then followed by flip UD, then followed by rotation of -45 and 45, then followed by another rotation of -90 and 90,\n and lastly followed by scaling with factor 0.5 and 1.5. '

In [9]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Загрузка файла с весами
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)

objects: ['Rails', 'Rails', 'Rails']
numids [1, 1, 1]
/content/drive/MyDrive/test_work/datasets/train/2534ia.jpg
objects: ['Rails', 'Rails', 'Rails']
numids [1, 1, 1]
/content/drive/MyDrive/test_work/datasets/train/2542ia.jpg
objects: ['Rails', 'Rails', 'Rails']
numids [1, 1, 1]
/content/drive/MyDrive/test_work/datasets/train/2546ia.jpg
objects: ['Rails', 'Rails', 'Rails']
numids [1, 1, 1]
/content/drive/MyDrive/test_work/datasets/train/2551ia.jpg
objects: ['Rails', 'Rails', 'Rails']
numids [1, 1, 1]
/content/drive/MyDrive/test_work/datasets/train/2558ia.jpg
objects: ['Rails', 'Rails', 'Rails']
numids [1, 1, 1]
/content/drive/MyDrive/test_work/datasets/train/2562ia.jpg
objects: ['Rails', 'Rails', 'Rails']
numids [1, 1, 1]
/content/drive/MyDrive/test_work/datasets/train/2568ia.jpg
objects: ['Rails']
numids [1]
/content/drive/MyDrive/test_work/datasets/train/2574ia.jpg
objects: ['Rails', 'Rails', 'Rails']
numids [1, 1, 1]
/content/drive/MyDrive/test_work/datasets/train/2578ia.jpg
objects

/usr/local/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training_2/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training_2/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training_2/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training_2/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training_2/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/Gathe

1/5 [=====>........................] - ETA: 4:00 - batch: 0.0000e+00 - size: 1.0000 - loss: 3.7233 - rpn_class_loss: 0.9178 - rpn_bbox_loss: 1.9386 - mrcnn_class_loss: 0.8670 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00

2023-07-20 14:21:56.945224: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-07-20 14:21:56.945286: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


2/5 [===========>..................] - ETA: 2:34 - batch: 0.5000 - size: 1.0000 - loss: 2.2607 - rpn_class_loss: 0.5436 - rpn_bbox_loss: 1.2270 - mrcnn_class_loss: 0.4901 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00    

2023-07-20 14:22:47.624510: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2023-07-20 14:22:47.624804: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1487] CUPTI activity buffer flushed
2023-07-20 14:22:47.653530: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2023-07-20 14:22:47.666938: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2023-07-20 14:22:47.701453: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /content/drive/MyDrive/test_work/logs/object20230720T1419/plugins/profile/2023_07_20_14_22_47
2023-07-20 14:22:47.720533: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /content/drive/MyDrive/test_work/logs/object20230720T1419/plugins/profile/2023_07_20_14_22_47/7ea35660606b.trace.json.gz
2023-07-20 14:22:47.746146: I ten

5/5 [==============================] - ETA: 0s - batch: 2.0000 - size: 1.0000 - loss: 2.4879 - rpn_class_loss: 0.3062 - rpn_bbox_loss: 0.9494 - mrcnn_class_loss: 0.2489 - mrcnn_bbox_loss: 0.6218 - mrcnn_mask_loss: 0.3616 

/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


5/5 [==============================] - 498s 110s/step - batch: 2.0000 - size: 1.0000 - loss: 2.4879 - rpn_class_loss: 0.3062 - rpn_bbox_loss: 0.9494 - mrcnn_class_loss: 0.2489 - mrcnn_bbox_loss: 0.6218 - mrcnn_mask_loss: 0.3616 - val_loss: 3.2926 - val_rpn_class_loss: 0.3354 - val_rpn_bbox_loss: 1.1606 - val_mrcnn_class_loss: 0.1112 - val_mrcnn_bbox_loss: 1.1266 - val_mrcnn_mask_loss: 0.5588
Epoch 2/250
5/5 [==============================] - 448s 99s/step - batch: 2.0000 - size: 1.0000 - loss: 2.7312 - rpn_class_loss: 0.0926 - rpn_bbox_loss: 0.8276 - mrcnn_class_loss: 0.1822 - mrcnn_bbox_loss: 1.0887 - mrcnn_mask_loss: 0.5401 - val_loss: 1.9920 - val_rpn_class_loss: 0.1582 - val_rpn_bbox_loss: 0.5594 - val_mrcnn_class_loss: 0.0792 - val_mrcnn_bbox_loss: 0.6411 - val_mrcnn_mask_loss: 0.5542
Epoch 3/250
5/5 [==============================] - 447s 98s/step - batch: 2.0000 - size: 1.0000 - loss: 1.9251 - rpn_class_loss: 0.0308 - rpn_bbox_loss: 0.5084 - mrcnn_class_loss: 0.0740 - mrcnn_bbox

IndexError: ignored